In [74]:
import json
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
land_df = pd.read_csv('land_area.csv')
popn_df = pd.read_csv('population.csv')

In [3]:
land_df.head()

,Areaname,STCOU,LND010190F,LND010190D,LND010190N1,LND010190N2,LND010200F,LND010200D,LND010200N1,LND010200N2,...,LND110210N1,LND110210N2,LND210190F,LND210190D,LND210190N1,LND210190N2,LND210200F,LND210200D,LND210200N1,LND210200N2
0,UNITED STATES,0,0,3787425.08,0,0,0,3794083.06,0,0,...,0,0,0,251083.35,0,0,0,256644.62,0,0
1,ALABAMA,1000,0,52422.94,0,0,0,52419.02,0,0,...,0,0,0,1672.71,0,0,0,1675.01,0,0
2,"Autauga, AL",1001,0,604.49,0,0,0,604.45,0,0,...,0,0,0,8.48,0,0,0,8.48,0,0
3,"Baldwin, AL",1003,0,2027.08,0,0,0,2026.93,0,0,...,0,0,0,430.55,0,0,0,430.58,0,0
4,"Barbour, AL",1005,0,904.59,0,0,0,904.52,0,0,...,0,0,0,19.59,0,0,0,19.61,0,0


In [4]:
popn_df.head()

,Areaname,STCOU,PST045200F,PST045200D,PST045200N1,PST045200N2,PST045201F,PST045201D,PST045201N1,PST045201N2,...,PST045207N1,PST045207N2,PST045208F,PST045208D,PST045208N1,PST045208N2,PST045209F,PST045209D,PST045209N1,PST045209N2
0,UNITED STATES,0,0,282171957,0,0,0,285081556,0,0,...,0,0,0,304374846,0,0,0,307006550,0,0
1,ALABAMA,1000,0,4451849,0,0,0,4464034,0,0,...,0,0,0,4677464,0,0,0,4708708,0,0
2,"Autauga, AL",1001,0,43872,0,0,0,44434,0,0,...,0,0,0,50354,0,0,0,50756,0,0
3,"Baldwin, AL",1003,0,141358,0,0,0,144988,0,0,...,0,0,0,176212,0,0,0,179878,0,0
4,"Barbour, AL",1005,0,29035,0,0,0,29223,0,0,...,0,0,0,29836,0,0,0,29737,0,0


In [5]:
land_df = land_df[['Areaname', 'LND010190D']]
popn_df = popn_df[['Areaname', 'PST045200D']]

In [6]:
lower_48 = ["AL", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", 
            "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
            "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
            "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
            "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

state_end = tuple(', ' + abbrev for abbrev in lower_48)

In [7]:
filtered_land_df = land_df[land_df.Areaname.str.endswith(state_end)]
filtered_popn_df = popn_df[land_df.Areaname.str.endswith(state_end)]

In [8]:
filtered_popn_df.shape

(3111, 2)

In [9]:
virginia_counties_df = filtered_land_df[filtered_land_df.Areaname.str.endswith(', VA')]

In [10]:
indices_to_delete = []
counties_set = set()
for index, row in virginia_counties_df.iterrows():
    county = row['Areaname']
    if county not in counties_set:
        counties_set.add(county)
    else:
        indices_to_delete.append(index)

In [11]:
filtered_land_df = filtered_land_df[~filtered_land_df.index.isin(indices_to_delete)]
filtered_popn_df = filtered_popn_df[~filtered_popn_df.index.isin(indices_to_delete)]

In [12]:
len(filtered_popn_df)

3106

In [13]:
combined_df = pd.merge(filtered_land_df, filtered_popn_df, on='Areaname', how='inner')

In [14]:
combined_df.shape

(3106, 3)

In [15]:
combined_df['state'] = combined_df.Areaname.str[-2:]

In [16]:
combined_df.head()

,Areaname,LND010190D,PST045200D,state
0,"Autauga, AL",604.49,43872,AL
1,"Baldwin, AL",2027.08,141358,AL
2,"Barbour, AL",904.59,29035,AL
3,"Bibb, AL",625.50,19936,AL
4,"Blount, AL",650.65,51181,AL


In [17]:
combined_df.rename(columns={'Areaname': 'county', 'LND010190D': 'area', 'PST045200D': 'popn'}, inplace=True)

In [18]:
combined_df.loc[combined_df.county == 'Broomfield, CO', 'area'] = 33.00

In [19]:
combined_df['density'] = combined_df['popn'] / combined_df['area']

In [20]:
state2pop = combined_df.groupby('state').agg({'popn': sum}).to_dict()['popn']
combined_df['state_popn'] = [state2pop[state] for state in combined_df.state]

In [21]:
combined_df.head()

,county,area,popn,state,density,state_popn
0,"Autauga, AL",604.49,43872,AL,72.576883,4451849
1,"Baldwin, AL",2027.08,141358,AL,69.734791,4451849
2,"Barbour, AL",904.59,29035,AL,32.097414,4451849
3,"Bibb, AL",625.50,19936,AL,31.872102,4451849
4,"Blount, AL",650.65,51181,AL,78.661339,4451849


In [22]:
state2density_metric = (combined_df.groupby('state').
                        apply(lambda x: round(x['popn'] * (x['density'] ** 1) / x['state_popn'], 1))
                        .groupby('state').sum()).to_dict()

In [23]:
sorted_density_metrics = sorted(list(state2density_metric.values()), reverse=True)

In [24]:
density_metric2state = {v: k for k, v in state2density_metric.items()}

In [25]:
ordered_density_metric2state = {x: density_metric2state[x] for x in sorted_density_metrics}

In [26]:
state_stats_df = pd.DataFrame(ordered_density_metric2state.keys(), columns=['density_metric'], 
                              index=ordered_density_metric2state.values())

In [27]:
state_stats_df.head()

,density_metric
NY,10711.4
NJ,2789.6
PA,1957.6
IL,1761.9
MD,1737.6


In [28]:
latlong_df = pd.read_csv('statelatlong.csv')

In [29]:
latlong_df.head()

,State,Latitude,Longitude,City
0,AL,32.601011,-86.680736,Alabama
1,AK,61.302501,-158.775020,Alaska
2,AZ,34.168219,-111.930907,Arizona
3,AR,34.751928,-92.131378,Arkansas
4,CA,37.271875,-119.270415,California


In [30]:
state_stats_df1 = (pd.merge(state_stats_df, latlong_df[['Latitude', 'State']],
                           left_index=True, right_on='State').drop(columns=['State']))
state_stats_df1.index = ordered_density_metric2state.values()

In [31]:
coastal_states = set('ME NH MA RI CT NY NJ PA MD DE VA NC SC GA FL WA OR CA'.split())

In [32]:
state_stats_df1['is_coastal'] = [state in coastal_states for state in state_stats_df.index]

In [33]:
state2passengers = {'NY': 50868391, 
                    'PA': 15285948 + 4670954 + 636916, 
                    'NJ': 19923009 + 589091,
                    'MD': 13371816,
                    'IL': round((83245472 / 2) + (22027737 / 2)),
                    'MA': 17759044,
                    'VA': 11470854 + 10596942 + 1777648 + 1602631,
                    'MO': 6793076 + 5391557 + 462126,
                    'CA': (39636042 + 25707101 + 10340164 + 5934639 + 5321603 + 5217242 
                           + 4969366 + 2104625 + 2077892 + 1386357 + 995801 + 761298),
                    'MI': 16847135 + 1334979 + 398508,
                    'CO': 28267394 + 657694,
                    'MN': 18123844,
                    'TX': 31283579 + 20062072 + 7554596 + 6285181 + 6095545 + 4179994 + 1414376,
                    'RI': 1803000,
                    'GA': 50501858 + 1056265,
                    'OH': 4083476 + 3567864 + 1019922 + 685553,
                    'CT': 2982194,
                    'IN': 4216766 + 360369 + 329957 + 204352,
                    'DE': 0,
                    'KY': 3269979 + 1631494 + 638316,
                    'FL': (20875813 + 20283541 + 14263270 + 9194994 + 4239261 + 3100624 + 2729129 
                           + 1321675 + 986766 + 915672 + 589860),
                    'NE': 2127387 + 162876,
                    'UT': 11143738,
                    'OR': 9071154,
                    'TN': 6338517 + 2016089 + 887103,
                    'LA': 5569705 + 364200,
                    'OK': 1796473 + 1342315,
                    'NC': 21511880 + 5401714 + 848261,
                    'KS': 781944,
                    'WA': 21887110 + 1570652,
                    'WI': 3496724 + 1043185 + 348026 + 314909,
                    'NH': 995403,
                    'AL': 1304467 + 527801 + 288209 + 173210,
                    'NM': 2341719,
                    'IA': 1216357 + 547786,
                    'AZ': 20896265 + 1594594 + 705731,
                    'SC': 1811695 + 991276 + 944849 + 553658,
                    'AR': 958824 + 673810,
                    'WV': 213412,
                    'ID': 1633507,
                    'NV': 22833267 + 1771864,
                    'ME': 886343 + 269013,
                    'MS': 491464 + 305157,
                    'VT': 593311,
                    'SD': 510105 + 272537,
                    'ND': 402976 + 273980 + 150634 + 132557 + 68829,
                    'MT': 553245 + 423213 + 381582 + 247816 + 176730 + 103239,
                    'WY': 342044 + 92805}


In [34]:
state_stats_df1['airport_arrivals'] = [state2passengers[state] for state in state_stats_df.index]

In [35]:
state_stats_df1.head()

,density_metric,Latitude,is_coastal,airport_arrivals
NY,10711.4,40.705626,True,50868391
NJ,2789.6,40.143006,True,20512100
PA,1957.6,40.994593,True,20593818
IL,1761.9,39.739318,False,52636604
MD,1737.6,38.806352,True,13371816


In [36]:
state_to_neighbors = {'AL': {'MS', 'TN', 'FL', 'GA', 'NC', 'SC'},
                      'GA': {'TN', 'FL', 'AL', 'SC', 'NC', 'MS'},
                      'FL': {'GA', 'AL', 'MS', 'SC'},
                      'MS': {'AL', 'TN', 'FL', 'LA', 'AR', 'GA'},
                      'LA': {'TX', 'AR', 'MS', 'OK', 'AL'},
                      'SC': {'FL', 'GA', 'NC', 'TN'},
                      'NC': {'SC', 'GA', 'TN', 'VA', 'KY'},
                      'AR': {'LA', 'TX', 'MS', 'TN', 'OK', 'MO', 'KY'},
                      'VA': {'NC', 'KY', 'WV', 'TN', 'DC', 'MD', 'DE'},
                      'DC': {'VA', 'MD', 'DE'},
                      'MD': {'DC', 'VA', 'WV', 'DE', 'NJ', 'PA'},
                      'DE': {'MD', 'DC', 'NJ', 'PA'},
                      'NJ': {'DE', 'MD', 'PA', 'NY', 'NJ', 'CT'},
                      'NY': {'NJ', 'PA', 'CT', 'MA', 'VT'},
                      'CT': {'NY', 'RI', 'MA'},
                      'RI': {'CT', 'MA'},
                      'MA': {'CT', 'RI', 'NH', 'VT', 'NY'},
                      'NH': {'VT', 'ME', 'MA'},
                      'ME': {'NH', 'MA', 'VT'},
                      'VT': {'NH', 'NY', 'MA'},
                      'PA': {'NY', 'NJ', 'MD', 'WV', 'OH', 'DE'},
                      'WV': {'DC', 'MD', 'PA', 'OH', 'KY', 'VA'},
                      'OH': {'PA', 'WV', 'MI', 'IN', 'KY'},
                      'MI': {'OH', 'WI', 'IN', 'IL'},
                      'KY': {'WV', 'OH', 'IN', 'IL', 'MO', 'TN', 'VA', 'AR', 'NC'},
                      'TN': {'KY', 'VA', 'NC', 'SC', 'GA', 'AL', 'MS', 'AR', 'MO', 'IL'},
                      'IN': {'KY', 'OH', 'MI', 'IL', 'WI'},
                      'IL': {'IN', 'MI', 'WI', 'IA', 'MO', 'KY', 'TN'},
                      'WI': {'IL', 'MN', 'MI', 'IA'},
                      'MN': {'MI', 'WI', 'IA', 'ND', 'SD', 'NE', 'IL'},
                      'IA': {'WI', 'MN', 'IL', 'MO', 'KS', 'NE', 'SD'},
                      'MO': {'IA', 'IL', 'KY', 'TN', 'AR', 'OK', 'KS', 'NE'},
                      'ND': {'SD', 'MN', 'MT', 'WY'},
                      'SD': {'ND', 'MN', 'IA', 'NE', 'MT', 'WY'},
                      'NE': {'SD', 'IA', 'MO', 'KS', 'WY', 'CO'},
                      'KS': {'NE', 'IA', 'MO', 'AR', 'OK', 'CO', 'TX', 'NM'},
                      'OK': {'KS', 'MO', 'AR', 'TX', 'NM', 'CO', 'LA'},
                      'TX': {'LA', 'AR', 'OK', 'NM', 'CO'},
                      'MT': {'ND', 'SD', 'WY', 'ID'},
                      'WY': {'MT', 'ND', 'SD', 'NE', 'CO', 'UT', 'ID'},
                      'CO': {'WY', 'NE', 'KS', 'OK', 'TX', 'NM', 'UT', 'AZ'},
                      'NM': {'CO', 'KS', 'OK', 'TX', 'AZ', 'UT'},
                      'ID': {'MT', 'WY', 'UT', 'NV', 'WA', 'OR'},
                      'UT': {'ID', 'WY', 'CO', 'NM', 'AZ', 'NV'},
                      'AZ': {'NM', 'CO', 'UT', 'NV', 'CA'},
                      'WA': {'ID', 'OR'},
                      'OR': {'WA', 'ID', 'NV', 'CA'},
                      'NV': {'ID', 'OR', 'UT', 'AZ', 'CA'},
                      'CA': {'OR', 'NV', 'AZ'},
                      }

state_to_neighbors = {k: list(v) for k, v in state_to_neighbors.items()}

In [37]:
age_df = pd.read_csv('age.csv')

In [38]:
abbrev2state = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

state2abbrev = {v: k for k, v in abbrev2state.items()}

In [39]:
age_df['Location'] = [state2abbrev[state] for state in age_df.Location]

In [40]:
state_stats_df2 = (pd.merge(state_stats_df1, age_df, left_index=True, right_on='Location')
                  .drop(columns=['Location']))
state_stats_df2.index = ordered_density_metric2state.values()

In [41]:
state_stats_df2.head()

,density_metric,Latitude,is_coastal,airport_arrivals,Children 0-18,Adults 19-25,Adults 26-34,Adults 35-54,Adults 55-64,65+
NY,10711.4,40.705626,True,50868391,0.22,0.09,0.13,0.26,0.14,0.16
NJ,2789.6,40.143006,True,20512100,0.23,0.08,0.11,0.27,0.14,0.16
PA,1957.6,40.994593,True,20593818,0.22,0.08,0.12,0.25,0.14,0.18
IL,1761.9,39.739318,False,52636604,0.24,0.09,0.12,0.26,0.13,0.15
MD,1737.6,38.806352,True,13371816,0.23,0.08,0.12,0.27,0.14,0.15


In [42]:
temps_df = pd.read_csv('temps.csv')

In [43]:
temps_df['State'] = [state2abbrev[state] for state in temps_df.State]

In [44]:
state_stats_df3 = (pd.merge(state_stats_df2, temps_df, left_index=True, right_on='State')
                  .drop(columns=['State']))
state_stats_df3.index = ordered_density_metric2state.values()

In [45]:
state_stats_df3.head()

,density_metric,Latitude,is_coastal,airport_arrivals,Children 0-18,Adults 19-25,Adults 26-34,Adults 35-54,Adults 55-64,65+,spring,summer,fall,winter
NY,10711.4,40.705626,True,50868391,0.22,0.09,0.13,0.26,0.14,0.16,43.6,66.5,48.1,23.3
NJ,2789.6,40.143006,True,20512100,0.23,0.08,0.11,0.27,0.14,0.16,50.6,72.2,54.8,33.0
PA,1957.6,40.994593,True,20593818,0.22,0.08,0.12,0.25,0.14,0.18,47.4,68.6,50.9,28.4
IL,1761.9,39.739318,False,52636604,0.24,0.09,0.12,0.26,0.13,0.15,51.6,73.4,53.8,28.3
MD,1737.6,38.806352,True,13371816,0.23,0.08,0.12,0.27,0.14,0.15,52.8,73.3,56.1,34.7


In [46]:
state2partisan_score = {
        'AL': 14,
        'AR': 15,
        'AZ': 5,
        'CA': -12,
        'CO': 1,
        'CT': -6,
        'DE': -6,
        'FL': 2,
        'GA': 5,
        'IA': 3,
        'ID': 19,
        'IL': -7,
        'IN': 9,
        'KS': 13,
        'KY': 15,
        'LA': 11,
        'MA': -12,
        'MD': -12,
        'ME': -3,
        'MI': -1,
        'MN': -1,
        'MO': 9,
        'MS': 9,
        'MT': 11,
        'NC': 3,
        'ND': 17,
        'NE': 14,
        'NH': 0,
        'NJ': -7,
        'NM': -3,
        'NV': -1,
        'NY': -12,
        'OH': 3,
        'OK': 20,
        'OR': -5,
        'PA': 0,
        'RI': -10,
        'SC': 8,
        'SD': 15,
        'TN': 14,
        'TX': 8,
        'UT': 20,
        'VA': -1,
        'VT': -15,
        'WA': -7,
        'WI': 0,
        'WV': 19,
        'WY': 25
}

In [47]:
state_stats_df3['partisan_score'] = [state2partisan_score[state] for state in state_stats_df3.index]

In [49]:
county_metrics_df = pd.read_csv('county_metrics.csv')

In [50]:
county_metrics_df.head()

,state,fipscode,county,population,hispanic,minority,female,unemployed,income,nodegree,bachelor,inactivity,obesity,density,cancer
0,Colorado,8117,Summit County,27239,15.173,4.918,45.996,2.5,68352,5.4,48.1,8.1,13.1,46.0,46.2
1,Colorado,8037,Eagle County,53653,30.040,5.169,47.231,3.1,76661,10.1,47.3,9.4,11.8,31.0,47.1
2,Idaho,16067,Minidoka County,19226,34.070,5.611,49.318,3.7,46332,24.1,11.8,18.3,34.2,80.0,61.8
3,Colorado,8113,San Miguel County,7558,10.154,4.747,46.808,3.7,59603,4.7,54.4,12.4,16.7,5.7,62.6
4,Utah,49051,Wasatch County,21600,13.244,4.125,48.812,3.4,65207,9.5,34.4,13.9,23.0,257.8,68.3


In [51]:
county_metrics_df['state'] = [state2abbrev[state] for state in county_metrics_df.state]

In [52]:
county_metrics_df = county_metrics_df[county_metrics_df.state.isin(lower_48)]

In [53]:
county_metrics_df.head()

,state,fipscode,county,population,hispanic,minority,female,unemployed,income,nodegree,bachelor,inactivity,obesity,density,cancer
0,CO,8117,Summit County,27239,15.173,4.918,45.996,2.5,68352,5.4,48.1,8.1,13.1,46.0,46.2
1,CO,8037,Eagle County,53653,30.040,5.169,47.231,3.1,76661,10.1,47.3,9.4,11.8,31.0,47.1
2,ID,16067,Minidoka County,19226,34.070,5.611,49.318,3.7,46332,24.1,11.8,18.3,34.2,80.0,61.8
3,CO,8113,San Miguel County,7558,10.154,4.747,46.808,3.7,59603,4.7,54.4,12.4,16.7,5.7,62.6
4,UT,49051,Wasatch County,21600,13.244,4.125,48.812,3.4,65207,9.5,34.4,13.9,23.0,257.8,68.3


In [54]:
state2pop_ = county_metrics_df.groupby('state').agg({'population': sum}).to_dict()['population']
county_metrics_df['state_popn'] = [state2pop_[state] for state in county_metrics_df.state]

In [55]:
county_metrics_df.head()

,state,fipscode,county,population,hispanic,minority,female,unemployed,income,nodegree,bachelor,inactivity,obesity,density,cancer,state_popn
0,CO,8117,Summit County,27239,15.173,4.918,45.996,2.5,68352,5.4,48.1,8.1,13.1,46.0,46.2,5022460
1,CO,8037,Eagle County,53653,30.040,5.169,47.231,3.1,76661,10.1,47.3,9.4,11.8,31.0,47.1,5022460
2,ID,16067,Minidoka County,19226,34.070,5.611,49.318,3.7,46332,24.1,11.8,18.3,34.2,80.0,61.8,1351143
3,CO,8113,San Miguel County,7558,10.154,4.747,46.808,3.7,59603,4.7,54.4,12.4,16.7,5.7,62.6,5022460
4,UT,49051,Wasatch County,21600,13.244,4.125,48.812,3.4,65207,9.5,34.4,13.9,23.0,257.8,68.3,2481585


In [56]:
state2hispanic = (county_metrics_df.groupby('state').
                  apply(lambda x: round((x['population'] * x['hispanic']) / x['state_popn'], 1))
                  .groupby('state').sum()).to_dict()

In [57]:
metrics = ['hispanic', 'minority', 'female', 'unemployed', 'income', 'nodegree', 'bachelor', 'inactivity',
          'obesity', 'cancer']

for metric in metrics:
    state2metric = (county_metrics_df.groupby('state').
                    apply(lambda x: round((x['population'] * x[metric]) / x['state_popn'], 3))
                    .groupby('state').sum()).to_dict()
    
    state_stats_df3[metric] = [state2metric[state] for state in state_stats_df3.index]

In [58]:
commuting_df_complete = pd.read_csv('commuting.csv')

In [59]:
commuting_df_complete.columns

Index(['State FIPS Code', 'County FIPS Code', 'State Name', 'County Name',
       'State FIPS Code.1', 'County FIPS Code.1', 'State Name.1',
       'County Name.1', 'Workers in Commuting Flow', ' Margin of Error',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13'],
      dtype='object')

In [60]:
commuting_df = commuting_df_complete[['State Name', 'State Name.1', 'Workers in Commuting Flow']]

In [61]:
commuting_df.rename(columns={'State Name': 'home_state', 
                             'State Name.1': 'work_state', 
                             'Workers in Commuting Flow': 'commuters'}, 
                   inplace=True)

/Users/wfried/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [62]:
lower_48_full_name = [abbrev2state[abbrev] for abbrev in lower_48]
commuting_df = commuting_df[commuting_df.work_state.isin(lower_48_full_name)]
                            # & (commuting_df.home_state != commuting_df.work_state)]

In [63]:
commuting_df['home_state'] = [state2abbrev[state] for state in commuting_df.home_state]
commuting_df['work_state'] = [state2abbrev[state] for state in commuting_df.work_state]

In [64]:
commuting_df.head(10)

,home_state,work_state,commuters
0,AL,AL,"8,828"
1,AL,AL,22
2,AL,AL,7
3,AL,AL,309
4,AL,AL,17
5,AL,AL,11
6,AL,AL,210
7,AL,AL,"2,244"
8,AL,AL,27
9,AL,AL,35


In [65]:
commuting_df['commuters'] = commuting_df['commuters'].apply(lambda x: int(''.join([y for y in x if y.isdigit()])))

In [66]:
commuting_groupby_df = (commuting_df.groupby(['work_state', 'home_state'], as_index=False)
                       .agg({'commuters': 'sum'}))

In [72]:
for work_state in state_stats_df3.index:
    vals = []
    for home_state in state_stats_df3.index:
        try:
            num1 = int((commuting_groupby_df[(commuting_groupby_df.work_state == work_state)
                       & (commuting_groupby_df.home_state == home_state)].commuters))
            num2 = int((commuting_groupby_df[(commuting_groupby_df.work_state == home_state)
                       & (commuting_groupby_df.home_state == work_state)].commuters))
            num = num1 + num2
            
        except TypeError:
            num = 0

        vals.append(num)

    state_stats_df3[work_state + '_dest'] = vals

In [71]:
state_stats_df3.to_csv('state_stats.csv')

with open('state2neighbors.csv', 'w') as f:
    f.write(json.dumps(state_to_neighbors))

In [75]:
state_stats_df3.head()

,density_metric,Latitude,is_coastal,airport_arrivals,Children 0-18,Adults 19-25,Adults 26-34,Adults 35-54,Adults 55-64,65+,spring,summer,fall,winter,partisan_score,hispanic,minority,female,unemployed,income,nodegree,bachelor,inactivity,obesity,cancer,NY_dest,NJ_dest,PA_dest,IL_dest,MD_dest,MA_dest,VA_dest,CA_dest,RI_dest,MI_dest,TX_dest,MO_dest,MN_dest,CT_dest,GA_dest,OH_dest,CO_dest,DE_dest,FL_dest,IN_dest,UT_dest,KY_dest,NE_dest,TN_dest,OR_dest,LA_dest,NC_dest,OK_dest,WA_dest,KS_dest,WI_dest,NH_dest,AZ_dest,SC_dest,AL_dest,IA_dest,NM_dest,WV_dest,NV_dest,AR_dest,ID_dest,ME_dest,MS_dest,VT_dest,SD_dest,ND_dest,MT_dest,WY_dest
NY,10711.4,40.705626,True,50868391,0.22,0.09,0.13,0.26,0.14,0.16,43.6,66.5,48.1,23.3,-12,18.737,29.756,51.436,5.347,61622.797,14.719,33.425,24.046,24.675,200.165,17663882,543903,60882,2776,4170,12995,3961,7401,1781,1941,3727,883,1100,119029,3106,3015,1194,1866,10725,991,473,529,285,1474,374,1197,3506,334,849,335,884,1379,914,1163,582,363,182,311,629,290,126,1139,390,7807,66,0,104,118
NJ,2789.6,40.143006,True,20512100,0.23,0.08,0.11,0.27,0.14,0.16,50.6,72.2,54.8,33.0,-7,19.372,27.143,51.199,5.676,72829.584,11.590,36.240,22.909,25.672,200.319,543903,7235854,250055,1647,4578,2924,2524,2672,497,883,2492,673,421,6800,1375,1967,777,14633,4597,527,185,395,119,624,201,363,1951,222,451,172,390,484,845,846,243,47,51,254,219,132,64,432,101,354,0,0,113,0
PA,1957.6,40.994593,True,20593818,0.22,0.08,0.12,0.25,0.14,0.18,47.4,68.6,50.9,28.4,0,6.767,17.371,51.067,5.199,54960.700,11.110,28.104,23.155,29.331,230.835,60882,250055,11145796,2549,73774,2970,5852,2925,370,2132,3607,1128,838,2099,2155,33473,825,62495,4658,1029,345,992,106,1283,204,814,2990,763,388,303,663,482,996,1156,724,278,206,22231,308,605,102,397,301,184,65,92,136,131
IL,1761.9,39.739318,False,52636604,0.24,0.09,0.12,0.26,0.13,0.15,51.6,73.4,53.8,28.3,-7,16.918,22.729,50.902,5.963,59266.824,12.397,32.078,21.199,27.340,225.471,2776,1647,2549,11554182,867,1135,1576,5237,243,7749,4876,108016,2609,752,2765,3942,1492,183,5709,101581,320,6576,896,2244,353,719,1944,598,925,822,65387,180,1427,529,626,55613,147,200,907,885,298,127,538,89,115,233,171,120
MD,1737.6,38.806352,True,13371816,0.23,0.08,0.12,0.27,0.14,0.15,52.8,73.3,56.1,34.7,-12,9.407,40.312,51.537,5.266,75296.062,11.089,37.129,21.545,28.940,225.486,4170,4578,73774,867,4880064,1447,196623,1433,239,661,1906,583,411,502,1622,1184,307,40943,3452,495,112,319,79,550,141,307,2677,58,509,128,305,144,160,662,309,69,33,28305,135,122,0,186,175,52,0,76,11,0
